In [81]:
import json

input_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered_landmarktype.jsonl"

def fix_landmark_type(triples):
    new_triples = []
    for triple in triples:
        if triple.get("rel") == "isLandmarkType":
            # 取地名类型前缀（如 "Rue"）
            sub = triple.get("sub", "")
            prefix = sub.split()[0] if sub else ""
            # 新 triple
            new_triples.append({
                "sub": prefix,
                "rel": "isLandmarkTypeOf",
                "obj": sub.lower()
            })
        else:
            new_triples.append(triple)
    return new_triples

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        data = json.loads(line)
        if "triples" in data:
            data["triples"] = fix_landmark_type(data["triples"])
        fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print("Traitement terminé !")

Traitement terminé !


In [82]:
import re
import json
from unidecode import unidecode
import dateparser

def extract_full_dates(text):
    """
    Extraire toutes les expressions de dates complètes dans la phrase (retourne une liste de segments d'origine, format français)
    """
    # Correspond à "1er février 1877", "2 mars 1912", "01 avril 1855", etc.
    pattern = r"\b(?:1er|\d{1,2})\s+[a-zéû]+(?:\s+\d{4})"
    results = []
    for m in re.finditer(pattern, text, re.IGNORECASE):
        phrase = text[m.start():m.end()]
        # On ne garde que celles qui peuvent être correctement analysées par dateparser
        dt = dateparser.parse(phrase, languages=["fr"])
        if dt:
            results.append(phrase.strip())
    return results

def replace_incomplete_dates(sent, triples):
    # Extraire toutes les dates complètes dans la phrase (ex : « 1er février 1877 »)
    full_dates = extract_full_dates(unidecode(sent))
    # Construire une table de correspondance { (année, mois): expression d'origine }
    mapping = {}
    for d in full_dates:
        dt = dateparser.parse(d, languages=["fr"])
        if dt:
            key = (str(dt.year), str(dt.month).zfill(2))
            mapping[key] = d.strip()

    # Traiter chaque triple individuellement
    for triple in triples:
        obj = triple.get("obj", "")
        m = re.fullmatch(r"(\d{4})-(\d{2})", obj)
        if m:
            year, month = m.group(1), m.group(2)
            if (year, month) in mapping:
                # Remplacer par l'expression temporelle réelle trouvée dans la phrase
                triple["obj"] = mapping[(year, month)]
        # Pour traiter aussi les formats yyyy-mm-dd si besoin
        m2 = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", obj)
        if m2:
            year, month, day = m2.group(1), m2.group(2), int(m2.group(3))
            for dstr, dphrase in mapping.items():
                if dstr == (year, month):
                    if str(day) in dphrase or f"{day:02d}" in dphrase or (day == 1 and "1er" in dphrase):
                        triple["obj"] = dphrase
    return triples

# ====== Traitement batch du fichier ======

input_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered_landmarktype.jsonl"    # Chemin du fichier d'entrée
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_fixed.jsonl"  # Chemin du fichier de sortie

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        d["triples"] = replace_incomplete_dates(d["sent"], d["triples"])
        fout.write(json.dumps(d, ensure_ascii=False) + "\n")

print("Traitement terminé !")


Traitement terminé !


In [83]:
import json
import dateparser
import re

def fr_date_to_iso(date_str):
    # Convertit une date en français (« 1er février 1877 », etc.) au format ISO (« 1877-02-01 »)
    dt = dateparser.parse(date_str, languages=['fr'])
    if dt:
        return dt.strftime("%Y-%m-%d")
    return date_str

def update_triples_date_obj(triples):
    # Pour chaque triple, si l'objet est une date complète en français, on la remplace par le format ISO
    pattern = r"\b(?:1er|\d{1,2})\s+[a-zéû]+(?:\s+\d{4})"
    for triple in triples:
        obj = triple.get("obj", "")
        # Vérifie si l'objet est une date en français (« 1er février 1877 », « 21 mai 1956 », etc.)
        if re.fullmatch(pattern, obj, re.IGNORECASE):
            iso_date = fr_date_to_iso(obj)
            triple["obj"] = iso_date
    return triples

# ========== Traitement batch du fichier JSONL ==========
input_file = "C:/Users/jguo/Desktop/PURE-main/data/test_fixed.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        d["triples"] = update_triples_date_obj(d["triples"])
        fout.write(json.dumps(d, ensure_ascii=False) + "\n")

print("Traitement terminé !")


Traitement terminé !


In [84]:
# prétraitement du jeu de donées pour supprimer des triplets raisonnés
#"sub": "Boulevard de Sébastopol", "rel": "hasGeometryChangeOn", "obj": "noTime"
# si le sujet et l'objet entre la relation "hasNewName" sont pareils, on supprime le triplet. {"sub": "Pont Alexandre III", "rel": "hasNewName", "obj": "pont Alexandre III"}
import json

# changer le chemin d'accès aux fichiers d'entrée et de sortie
input_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered.jsonl"

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        data = json.loads(line)
        if "triples" in data:
            new_triples = []
            for triple in data["triples"]:
                rel = triple.get("rel", "")
                obj = triple.get("obj", "")
                sub = triple.get("sub", "")

                # 1：sauter obj == noTime
                if obj == "noTime":
                    continue
                # 2：sauter hasNewName et sub.lower() == obj.lower()
                if rel == "hasNewName" and sub.strip().lower() == obj.strip().lower():
                    continue
                if rel == "hasOldName" and sub.strip().lower() == obj.strip().lower():
                    continue

                # garder ce triplet
                new_triples.append(triple)

            # actualiser les triplets
            data["triples"] = new_triples

        # Chaque ligne de sortie reste un dictionnaire JSON.
        fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print(" Tous les filtres ont été appliqués avec succès. Le fichier de sortie a été enregistré à l’emplacement suivant ：", output_file)

 Tous les filtres ont été appliqués avec succès. Le fichier de sortie a été enregistré à l’emplacement suivant ： C:/Users/jguo/Desktop/PURE-main/data/test_filtered.jsonl


In [1]:
### générer le fichier de rel2id.json
import json
from collections import OrderedDict

input_path = "C:/Users/jguo/Documents/BERT-LLM/dataset/pegazus-event-extraction/new_train.jsonl"     # 替换为你的输入 JSONL 路径
output_path = "C:/Users/jguo/Documents/BERT-LLM/dataset/pegazus-event-extraction/rel2id.json"   # 输出文件

rels = set()

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        for t in obj.get("triples", []):
            rel = (t.get("rel") or "").strip()
            if rel:
                rels.add(rel)

# 稳定且可复现：按字母序排序
rel_list = sorted(rels)

rel2id = OrderedDict((rel, idx) for idx, rel in enumerate(rel_list))

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(rel2id, f, ensure_ascii=False, indent=2)

print(f"Collected {len(rel2id)} relations. Saved to {output_path}")

Collected 12 relations. Saved to C:/Users/jguo/Documents/BERT-LLM/dataset/pegazus-event-extraction/rel2id.json
